<a href="https://colab.research.google.com/github/nyny137/cynthia/blob/main/Modelling_for_Social_Media_Ads_Booking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Intall and Import Package/Library
!pip install pulp
import pulp
!apt-get install -y -qq glpk-utils
from pulp import GLPK
from pulp import LpVariable, LpProblem, LpMaximize, LpMinimize, lpSum

In [ ]:
prob = pulp.LpProblem("Maximize_Impressions", pulp.LpMinimize)

In [ ]:
from pulp import LpMaximize, LpProblem, LpVariable, LpStatus

In [ ]:
from pulp import LpMaximize, LpProblem, LpVariable, LpStatus

# Define platform characteristics
platforms = ['Youtube', 'TikTok', 'Instagram']
active_users = {'Micro': 50005, 'Mid': 300000, 'Macro': 750000}
kol_type = ['Micro', 'Mid', 'Macro']
impression_rate = {
    ('Youtube', 'Micro'): 0.019,
    ('Youtube', 'Mid'): 0.0347,
    ('Youtube', 'Macro'): 0.034,
    ('TikTok', 'Micro'): 0.124,
    ('TikTok', 'Mid'): 0.109,
    ('TikTok', 'Macro'): 0.108,
    ('Instagram', 'Micro'): 0.0106,
    ('Instagram', 'Mid'): 0.0091,
    ('Instagram', 'Macro'): 0.0086}
cost = {
    ('Youtube', 'Micro'): 600,
    ('Youtube', 'Mid'): 5500,
    ('Youtube', 'Macro'): 15000,
    ('TikTok', 'Micro'): 75,
    ('TikTok', 'Mid'): 688,
    ('TikTok', 'Macro'): 1875,
    ('Instagram', 'Micro'): 219,
    ('Instagram', 'Mid'): 775,
    ('Instagram', 'Macro'): 3138}

# demographics across platforms
demographics = {
     'Youtube': {'Income': 0.34, 'Age18-24': 0.155, 'CollegeEd': 0.37},
    'TikTok': {'Income': 0.31, 'Age18-24': 0.362, 'CollegeEd': 0.36},
    'Instagram': {'Income': 0.32, 'Age18-24': 0.308, 'CollegeEd': 0.44},
}


In [ ]:
#the percentage of target users on per platform
demographic_factor_youtube = 0.33 * (demographics['Youtube']['Income'] + demographics['Youtube']['Age18-24'] + demographics['Youtube']['CollegeEd'])
demographic_factor_tiktok = 0.33 * (demographics['TikTok']['Income'] + demographics['TikTok']['Age18-24'] + demographics['TikTok']['CollegeEd'])
demographic_factor_instagram = 0.33 * (demographics['Instagram']['Income'] + demographics['Instagram']['Age18-24'] + demographics['Instagram']['CollegeEd'])

#the reach on per platform
# Youtube
k11 = demographic_factor_youtube * active_users['Micro'] * impression_rate[('Youtube', 'Micro')]
k21 = demographic_factor_youtube * active_users['Mid'] * impression_rate[('Youtube', 'Mid')]
k31 = demographic_factor_youtube * active_users['Macro'] * impression_rate[('Youtube', 'Macro')]

# TikTok
k12 = demographic_factor_tiktok * active_users['Micro'] * impression_rate[('TikTok', 'Micro')]
k22 = demographic_factor_tiktok * active_users['Mid'] * impression_rate[('TikTok', 'Mid')]
k32 = demographic_factor_tiktok * active_users['Macro'] * impression_rate[('TikTok', 'Macro')]

# Instagram
k13 = demographic_factor_instagram * active_users['Micro'] * impression_rate[('Instagram', 'Micro')]
k23 = demographic_factor_instagram * active_users['Mid'] * impression_rate[('Instagram', 'Mid')]
k33 = demographic_factor_instagram * active_users['Macro'] * impression_rate[('Instagram', 'Macro')]


In [ ]:
(k11,k21,k31,k12,k22,k32,k13,k23,k33)

(271.20461775000007,
 2971.5345000000007,
 7278.975000000001,
 2111.6831472000003,
 11136.312000000002,
 27585.360000000004,
 186.81187932,
 962.1612000000002,
 2273.238)

In [ ]:
k_value = {
    'Youtube,Micro': k11,
    'Youtube,Mid': k21,
    'Youtube,Macro': k31,
    'TikTok,Micro': k12,
    'TikTok,Mid': k22,
    'TikTok,Macro': k32,
    'Instagram,Micro': k13,
    'Instagram,Mid': k23,
    'Instagram,Macro': k33
}

In [ ]:
#Decision variables
ads = pulp.LpVariable.dicts("Ads", [(platform, kol_type) for platform in platforms for kol_type in ['Micro', 'Mid', 'Macro']], lowBound=0, upBound=25, cat='Integer')


# Deviation variables for each platform-KOL combination
d1_pos = pulp.LpVariable("d1_pos", lowBound=0, upBound=None, cat='Continuous')
d1_neg = pulp.LpVariable("d1_neg", lowBound=0, upBound=None, cat='Continuous')
d2_pos = pulp.LpVariable("d2_pos", lowBound=0, upBound=None, cat='Continuous')
d2_neg = pulp.LpVariable("d2_neg", lowBound=0, upBound=None, cat='Continuous')
d3_pos = pulp.LpVariable("d3_pos", lowBound=0, upBound=None, cat='Continuous')
d3_neg = pulp.LpVariable("d3_neg", lowBound=0, upBound=None, cat='Continuous')
d4_pos = pulp.LpVariable("d4_pos", lowBound=0, upBound=None, cat='Continuous')
d4_neg = pulp.LpVariable("d4_neg", lowBound=0, upBound=None, cat='Continuous')


In [ ]:
# Add Objective Function to the Environment
prob += d4_pos

In [ ]:
# Add Soft Constraints Function to the Environment
# Goal 1 --- Impression
prob += sum([k_value[platform + ',' + kol] * ads[(platform, kol)] for platform in platforms for kol in kol_type]) - d1_pos + d1_neg == 220000


# Goal 2 --- The cost for Instagram
prob += sum(ads['Instagram', kol_type] * cost[('Instagram', kol_type)] for kol_type in ['Micro', 'Mid', 'Macro']) - d2_pos + d2_neg == 33000 #30%

# Goal 3 --- The cost for TikTok
prob += sum(ads['TikTok', kol_type] * cost[('TikTok', kol_type)]  for kol_type in ['Micro', 'Mid', 'Macro']) - d3_pos + d3_neg == 55000   #50%

# Goal 4 --- The cost for Youtube
prob += sum(ads['Youtube', kol_type] * cost[('Youtube', kol_type)]  for kol_type in ['Micro', 'Mid', 'Macro']) - d4_pos + d4_neg == 22000  #20%

prob += d1_neg == 0
prob += d2_pos == 0
prob += d3_pos == 0



In [ ]:
# Solve the Problem
prob.writeLP("Maximize_Impressions.lp")
prob.solve(GLPK(msg=True, options=['--ranges', 'sensitivity.txt']))

# The Status of the Solution
print("Model Status:{}".format(pulp.LpStatus[prob.status]))

# To Display Optimal Decision Variables & Reduced Cost per Variable
for v in prob.variables():
	  print(v.name, "=", v.varValue)

# To Display Optimal Value of Objective Function
print("Objective=", pulp.value(prob.objective))

Model Status:Optimal
Ads_('Instagram',_'Macro') = 0
Ads_('Instagram',_'Micro') = 0
Ads_('Instagram',_'Mid') = 25
Ads_('TikTok',_'Macro') = 5
Ads_('TikTok',_'Micro') = 24
Ads_('TikTok',_'Mid') = 0
Ads_('Youtube',_'Macro') = 1
Ads_('Youtube',_'Micro') = 1
Ads_('Youtube',_'Mid') = 0
d1_neg = 0.0
d1_pos = 211.405
d2_neg = 13625.0
d2_pos = 0.0
d3_neg = 43825.0
d3_pos = 0.0
d4_neg = 6400.0
d4_pos = 0.0
Objective= 0.0
